In [1]:
#import the required Python packages

import pandas as pd
import datetime   
#import pyeto
import numpy as np
import ast       
from ast import literal_eval  
from pandas import DataFrame

#from pyeto import fao
from datetime import datetime
%matplotlib inline     

In [2]:
#Note: You might need to add a column for tds level:

df=pd.read_excel('Pilot20190124_Part3.xlsx') 


In [3]:
df.columns = map(str.lower, df.columns)

In [4]:
df['tds']

0        3000
1        3000
2        3000
3        3000
4        3000
5        3000
6        3000
7        3000
8        3000
9        3000
10       3000
11       3000
12       3000
13       3000
14       3000
15       3000
16       3000
17       3000
18       3000
19       3000
20       3000
21       3000
22       3000
23       3000
24       3000
25       3000
26       3000
27       3000
28       3000
29       3000
         ... 
29916    2000
29917    2000
29918    2000
29919    2000
29920    2000
29921    2000
29922    2000
29923    2000
29924    2000
29925    2000
29926    2000
29927    2000
29928    2000
29929    2000
29930    2000
29931    2000
29932    2000
29933    2000
29934    2000
29935    2000
29936    2000
29937    2000
29938    2000
29939    2000
29940    2000
29941    2000
29942    2000
29943    2000
29944    2000
29945    2000
Name: tds, Length: 29946, dtype: int64

In [5]:
#Set the default values
for i in range (1,13):
    df['osmotic_pressure'.format(i)] = 0
    df['Edesal_GWh_{}'.format(i)] = 0 

In [6]:
#Define inputs:

osm_coeff = 0.95 #osmotic coefficient =1 
molar_mass = 58.44 # molar mass of NaCl = 58.44 mg/mol
dissociation = 2 #NaCL dissociation is to 2 ions Na and Cl
concentration = df['tds'] 
gas_const = 0.083145  #universal gas constant (R) = 0.083145
wtemp = 25 + 273 #water temperature in Kalven (Tc+273): 25 degree is an assumption


#solute_concentration = dissociation * concentration / molar_mass

#osmotic_pressure = osm_coeff * solute_concentration * gas_const * temp
 


#if (df['tds'] > 2000):
#    return df['osmotic_pressure'] = osm_coeff * (dissociation *df['tds']/molar_mass) *  gas_const * wtemp
#else:
#    return df['osmotic_pressure'] = 0

df['osmotic_pressure'] = osm_coeff * (dissociation *df.tds[df['tds']>2000]/molar_mass) *  gas_const * wtemp

df['osmotic_pressure'] = df['osmotic_pressure'].fillna(0)

#df.tds[df['tds']>2000]: This set the criteria that for tds>2000 desalination required

In [7]:
#Energy intensity in Kwh/m3

df['Einten_KWh/m3'] = df['osmotic_pressure']/36

In [8]:
%%time

for i in range(1,13):
    Edesal_GWh = 'Edesal_GWh_{}'.format(i)
    SSWD = 'sswd_{}'.format(i)
    
    
    df[Edesal_GWh] = (df['Einten_KWh/m3'] * df[SSWD]) /1000000
    
    

Wall time: 28 ms


In [9]:
#Filter data by provenence: 
df2=df.groupby('prov', as_index=True).sum()
sswd = df.filter(like='sswd_').sum()
Edesal = df.filter(like='Edesal_GWh_').sum()

final = df2.filter(['area_m2','sswd_','osmotic_pressure','Edesal_GWh_','country'])
print(final)

                     area_m2  osmotic_pressure
prov                                          
Adrar           2.411980e+08      1.765135e+07
Jabal Algharbi  8.506696e+07      1.123509e+07
Tozeur          1.265473e+08      5.377087e+06


In [10]:
#Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Pilot2019024_Part4.xlsx', engine='xlsxwriter')
writer.book.use_zip64()

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='All')
df2.to_excel(writer, sheet_name='Prov')
#sswd.to_excel(writer, sheet_name='SSWD in m3')
#Edesal.to_excel(writer, sheet_name='Edesal in KWh')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [1]:
import sys
sys.version

'3.5.3 |Anaconda 4.4.0 (64-bit)| (default, May 15 2017, 10:43:23) [MSC v.1900 64 bit (AMD64)]'